In [1]:
!pip install -r requirement.txt

You should consider upgrading via the '/Users/rajveeryadav/Desktop/Vistora/venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import snowflake.connector

In [3]:
import os
from dotenv import load_dotenv

Load the SnowFlake credentials like account details

In [4]:
load_dotenv(".env")

user_name = os.getenv('SNOW_USER')
password = os.getenv('SNOW_PASS')
account = os.getenv('SNOW_ACCOUNT')
warehouse = os.getenv('warehouse')
snow_database = os.getenv('database')
snow_schema = os.getenv('schema')

Create a connection

In [5]:
conn = snowflake.connector.connect(
    user=user_name,
    password=password,
    account=account,
    warehouse=warehouse,
    database=snow_database,
    schema=snow_schema
)

# Fetching the raw data from SnowFlake using SQL and pandas

In [6]:
import pandas as pd

In [7]:
# fetching and preprocessing table sql script
with open('snow_preprocessing_script.sql', 'r') as file:
    sql_query = file.read()

In [8]:
df = pd.read_sql(sql_query, conn)

/var/folders/cz/lky96vlx5b3gt97txms5jymm0000gn/T/ipykernel_99073/4280099553.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [9]:
df.head()

,C_CUSTOMER_ID,FIRST_NAME_CLEAN,LAST_NAME_CLEAN,IS_PREFERRED_CUST,C_BIRTH_YEAR,C_BIRTH_MONTH,C_BIRTH_DAY,AGE,BIRTH_SEASON,AGE_GROUP,BIRTH_COUNTRY_CLEAN,EMAIL_SAFE,HAS_LOGIN,LAST_REVIEW_DATE,LAST_REVIEW_YEAR,DAYS_SINCE_LAST_REVIEW
0,AAAAAAAABNJALGAA,DAVID,OLSEN,1,1942.0,6.0,8.0,82.0,3,3,SERBIA,David.Olsen@f.com,0,1970-01-29,1970.0,20267.0
1,AAAAAAAACNJALGAA,IONA,WHITE,1,1952.0,5.0,25.0,72.0,2,3,THAILAND,Iona.White@8PZ6.org,0,1970-01-29,1970.0,20267.0
2,AAAAAAAADNJALGAA,JAMES,MENDOZA,0,1931.0,5.0,17.0,93.0,2,3,EQUATORIAL GUINEA,James.Mendoza@hSAR30xkQc.com,0,1970-01-29,1970.0,20267.0
3,AAAAAAAAENJALGAA,None,HUNTER,0,NaN,NaN,NaN,NaN,0,3,None,Bessie.Hunter@SEg5Zt3Inh7.edu,0,None,NaN,NaN
4,AAAAAAAAFNJALGAA,ROBERT,NELSON,0,1961.0,11.0,18.0,63.0,4,3,URUGUAY,Robert.Nelson@lzzdMmJ.com,0,1970-01-29,1970.0,20267.0


In [10]:
df.shape

(1000, 16)

lets do some analysis on dataframe values

In [11]:
# Lets see how many NaN values are there in each column of the data we got
df.isnull().sum()

C_CUSTOMER_ID              0
FIRST_NAME_CLEAN          33
LAST_NAME_CLEAN           34
IS_PREFERRED_CUST          0
C_BIRTH_YEAR              39
C_BIRTH_MONTH             29
C_BIRTH_DAY               32
AGE                       39
BIRTH_SEASON               0
AGE_GROUP                  0
BIRTH_COUNTRY_CLEAN       31
EMAIL_SAFE                 0
HAS_LOGIN                  0
LAST_REVIEW_DATE          30
LAST_REVIEW_YEAR          30
DAYS_SINCE_LAST_REVIEW    30
dtype: int64

In [12]:
# we will drop these values 
df_clean = df.dropna()
df_clean.shape # we had 1000 rows of data

(937, 16)

We will check what kind of data values are there in the dataframe
as we can se in the data types desciption below there are some objects and most of them are numeric data type.
There is a BIRTH_COUNTRY_CLEAN column we can map the values to an interger, remove the names columns, Customer id the LAST_REVIEW_DATE column 

In [13]:
df_clean.dtypes

C_CUSTOMER_ID              object
FIRST_NAME_CLEAN           object
LAST_NAME_CLEAN            object
IS_PREFERRED_CUST           int64
C_BIRTH_YEAR              float64
C_BIRTH_MONTH             float64
C_BIRTH_DAY               float64
AGE                       float64
BIRTH_SEASON                int64
AGE_GROUP                   int64
BIRTH_COUNTRY_CLEAN        object
EMAIL_SAFE                 object
HAS_LOGIN                   int64
LAST_REVIEW_DATE           object
LAST_REVIEW_YEAR          float64
DAYS_SINCE_LAST_REVIEW    float64
dtype: object

In [14]:
df_clean['BIRTH_COUNTRY_CODE'], unique = pd.factorize(df_clean['BIRTH_COUNTRY_CLEAN'])

/var/folders/cz/lky96vlx5b3gt97txms5jymm0000gn/T/ipykernel_99073/2159245035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['BIRTH_COUNTRY_CODE'], unique = pd.factorize(df_clean['BIRTH_COUNTRY_CLEAN'])


In [15]:
df_clean.head()

,C_CUSTOMER_ID,FIRST_NAME_CLEAN,LAST_NAME_CLEAN,IS_PREFERRED_CUST,C_BIRTH_YEAR,C_BIRTH_MONTH,C_BIRTH_DAY,AGE,BIRTH_SEASON,AGE_GROUP,BIRTH_COUNTRY_CLEAN,EMAIL_SAFE,HAS_LOGIN,LAST_REVIEW_DATE,LAST_REVIEW_YEAR,DAYS_SINCE_LAST_REVIEW,BIRTH_COUNTRY_CODE
0,AAAAAAAABNJALGAA,DAVID,OLSEN,1,1942.0,6.0,8.0,82.0,3,3,SERBIA,David.Olsen@f.com,0,1970-01-29,1970.0,20267.0,0
1,AAAAAAAACNJALGAA,IONA,WHITE,1,1952.0,5.0,25.0,72.0,2,3,THAILAND,Iona.White@8PZ6.org,0,1970-01-29,1970.0,20267.0,1
2,AAAAAAAADNJALGAA,JAMES,MENDOZA,0,1931.0,5.0,17.0,93.0,2,3,EQUATORIAL GUINEA,James.Mendoza@hSAR30xkQc.com,0,1970-01-29,1970.0,20267.0,2
4,AAAAAAAAFNJALGAA,ROBERT,NELSON,0,1961.0,11.0,18.0,63.0,4,3,URUGUAY,Robert.Nelson@lzzdMmJ.com,0,1970-01-29,1970.0,20267.0,3
5,AAAAAAAAGNJALGAA,LOIS,JONES,1,1983.0,5.0,23.0,41.0,2,2,KYRGYZSTAN,Lois.Jones@lvD.org,0,1970-01-29,1970.0,20267.0,4


In [16]:
df_clean.dtypes

C_CUSTOMER_ID              object
FIRST_NAME_CLEAN           object
LAST_NAME_CLEAN            object
IS_PREFERRED_CUST           int64
C_BIRTH_YEAR              float64
C_BIRTH_MONTH             float64
C_BIRTH_DAY               float64
AGE                       float64
BIRTH_SEASON                int64
AGE_GROUP                   int64
BIRTH_COUNTRY_CLEAN        object
EMAIL_SAFE                 object
HAS_LOGIN                   int64
LAST_REVIEW_DATE           object
LAST_REVIEW_YEAR          float64
DAYS_SINCE_LAST_REVIEW    float64
BIRTH_COUNTRY_CODE          int64
dtype: object

In [17]:
df_clean.shape

(937, 17)

In [18]:
# Dropping the columns that are string or object 
df_final = df_clean.drop(columns=df_clean.select_dtypes(include=['object']).columns)
# except the C_CUSTOMER_ID
df_final['CUSTOMER_ID'] = df_clean['C_CUSTOMER_ID']

In [19]:
df_final

,IS_PREFERRED_CUST,C_BIRTH_YEAR,C_BIRTH_MONTH,C_BIRTH_DAY,AGE,BIRTH_SEASON,AGE_GROUP,HAS_LOGIN,LAST_REVIEW_YEAR,DAYS_SINCE_LAST_REVIEW,BIRTH_COUNTRY_CODE,CUSTOMER_ID
0,1,1942.0,6.0,8.0,82.0,3,3,0,1970.0,20267.0,0,AAAAAAAABNJALGAA
1,1,1952.0,5.0,25.0,72.0,2,3,0,1970.0,20267.0,1,AAAAAAAACNJALGAA
2,0,1931.0,5.0,17.0,93.0,2,3,0,1970.0,20267.0,2,AAAAAAAADNJALGAA
4,0,1961.0,11.0,18.0,63.0,4,3,0,1970.0,20267.0,3,AAAAAAAAFNJALGAA
5,1,1983.0,5.0,23.0,41.0,2,2,0,1970.0,20267.0,4,AAAAAAAAGNJALGAA
...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1978.0,9.0,24.0,46.0,4,2,0,1970.0,20267.0,120,AAAAAAAAELNALGAA
996,1,1974.0,2.0,28.0,50.0,1,3,0,1970.0,20267.0,21,AAAAAAAAFLNALGAA
997,0,1938.0,3.0,13.0,86.0,2,3,0,1970.0,20267.0,164,AAAAAAAAGLNALGAA
998,0,1961.0,9.0,6.0,63.0,4,3,0,1970.0,20267.0,49,AAAAAAAAHLNALGAA


In [20]:
df_final.columns

Index(['IS_PREFERRED_CUST', 'C_BIRTH_YEAR', 'C_BIRTH_MONTH', 'C_BIRTH_DAY',
       'AGE', 'BIRTH_SEASON', 'AGE_GROUP', 'HAS_LOGIN', 'LAST_REVIEW_YEAR',
       'DAYS_SINCE_LAST_REVIEW', 'BIRTH_COUNTRY_CODE', 'CUSTOMER_ID'],
      dtype='object')

# Storing the data in SnowFlake Feature Store

In [21]:
from snowflake.snowpark import Session

connection_parameters = {
    "user": user_name,
    "password": password,
    "account": account,
    "warehouse": warehouse,
    "database": "VISTORA",
    "schema": "VISTORA_SCHEMA"
}

session = Session.builder.configs(connection_parameters).create()

In [22]:
from snowflake.ml.feature_store import FeatureStore, CreationMode

In [23]:
from snowflake.connector.pandas_tools import write_pandas

In [24]:

fs = FeatureStore(
    session=session,
    database="VISTORA",
    name="VISTORA_SCHEMA",    
    default_warehouse=warehouse,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST  # Creates if it doesn't exist
)


In [25]:
from snowflake.ml.feature_store import Entity

entity = Entity(name="customer", join_keys=["CUSTOMER_ID"], desc="Unique customer identifier")
fs.register_entity(entity)


/Users/rajveeryadav/Desktop/Vistora/venv/lib/python3.9/site-packages/snowflake/ml/feature_store/feature_store.py:186: UserWarning: Entity CUSTOMER already exists. Skip registration.
  return f(self, *args, **kargs)


Entity(name=CUSTOMER, join_keys=['CUSTOMER_ID'], owner=None, desc=Unique customer identifier)

In [26]:
write_pandas(conn, df_final, 'VISTORA_FEATURE_TABLE', database=connection_parameters["database"], schema=connection_parameters["schema"], overwrite=True)

/var/folders/cz/lky96vlx5b3gt97txms5jymm0000gn/T/ipykernel_99073/3570353212.py:1: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(conn, df_final, 'VISTORA_FEATURE_TABLE', database=connection_parameters["database"], schema=connection_parameters["schema"], overwrite=True)


(True,
 1,
 937,
 [('snowpark_temp_stage_k3hsvgothe/file0.txt',
   'LOADED',
   937,
   937,
   1,
   0,
   None,
   None,
   None,
   None)])